# 03 Import and cleanse investors data

## Import modules

In [1]:
import pandas as pd
import numpy as np
import os
pd.options.display.max_columns = 60

## Import raw data

In [2]:
# Find file location
!ls ../../data/raw/

Datasheet_Cleansed.xlsx Datasheet_Raw.xlsx


In [3]:
file_path = '../../data/raw/Datasheet_Cleansed.xlsx'
df = pd.read_excel(file_path, sheet_name='Funding_Clean')

df.head()

,Deal ID,Deal Date,Deal Y,Firm,Unique ID Lookup,Round Total ($),Round Total,# of investors,Avg/Investor,Participating Investor,Deal Type,Country,Head office,Year founded,CodeX Category
0,DEAL0001,2000-03-22,2000,LexisNexis,FIRM0002,30000000,30000000.0,5,6000000.0,Actua Corporation,Venture Capital,USA,New York,1973,Legal Research
1,DEAL0001,2000-03-22,2000,LexisNexis,FIRM0002,above,30000000.0,5,6000000.0,Commerce One,Venture Capital,USA,New York,1973,Legal Research
2,DEAL0001,2000-03-22,2000,LexisNexis,FIRM0002,above,30000000.0,5,6000000.0,Kaplan,Venture Capital,USA,New York,1973,Legal Research
3,DEAL0001,2000-03-22,2000,LexisNexis,FIRM0002,above,30000000.0,5,6000000.0,Schoolhouse Partners,Venture Capital,USA,New York,1973,Legal Research
4,DEAL0001,2000-03-22,2000,LexisNexis,FIRM0002,above,30000000.0,5,6000000.0,WR Hambrecht,Venture Capital,USA,New York,1973,Legal Research


## Clean up headers for easier access

In [4]:
[col.lower() for col in df.columns]

['deal id',
 'deal date',
 'deal y',
 'firm',
 'unique id lookup',
 'round total ($)',
 'round total',
 '# of investors',
 'avg/investor',
 'participating investor',
 'deal type',
 'country',
 'head office',
 'year founded',
 'codex category']

In [5]:
cols = ['deal_id','deal_date','deal_year','firm_name','firm_id','round_total_raw','round_total','num_investors',
        'avg_invest_per_investor','investor_name','deal_type','country','city','year_founded','codex_category']
df.columns = cols

## Any nulls or missing data?

In [6]:
df.isnull().sum()

deal_id                      0
deal_date                    0
deal_year                    0
firm_name                    0
firm_id                      0
round_total_raw              0
round_total                  0
num_investors                0
avg_invest_per_investor      0
investor_name              191
deal_type                    0
country                      0
city                         0
year_founded                 0
codex_category               0
dtype: int64

**Notes:** It looks like 191 of the investor names are not known.  Instead of NaNs, let's replace with "undisclosed"

In [7]:
# Set Nan values for investor name to "undisclosed"
df.loc[df.investor_name.isnull(), 'investor_name'] = 'undisclosed'

**Notes:** Let's first think through what we're doing here.  The investor data is the most granular because it has a many-to-one relationship with deals.  The purpose of the dataset that we're going to put together here will be for doing analysis at the investor level.  If we want to look at the data at a less granular data, we should a different data set that just joins firms with deals

The raw investors data already has all the deals data already, so we can ignore that and just first join the firms data

## Fix typo:
- Firm_name:  Lawyaw
- Error:  firm_id = FIRM0657
- Fix:  firm_id = FIRM0843	

In [8]:
df.loc[df.firm_name == 'Lawyaw', 'firm_id'] = 'FIRM0843'

## Fix typos:
- ``Seed Fund`` sometimes spelled as ``Seed fund``

In [9]:
df.loc[df.deal_type == 'Seed fund', 'deal_type'] = 'Seed Fund'

## Pull in cleansed firms data and merge into our investors data

### Import firms data

In [10]:
folder_path = '../../data/interim/'
firms_path = os.path.join(folder_path, 'firms.csv')

firms = pd.read_csv(firms_path,index_col = 0)

### Create ``Firms_join`` dataframe

In [11]:
firms_cols_keep = ['firm_id','market','problem_1','problem_2','problem_3','analytics','ai','point_integrated']
firms_join = firms[firms_cols_keep]

### Merge selected firms columns with investor data

In [12]:
df = df.merge(firms_join, how='left', on='firm_id')

**Note:** I want to double check if any typos in investor name, since I don't have a key for it

## Add some feature engineering: firm_age at funding

In [13]:
df['firm_age'] = (df.deal_year - df.year_founded)

# Some of the firm age shows negative.  Set to 0
df.loc[df.firm_age <0, 'firm_age'] = 0

## Check for Nulls

In [14]:
df.isnull().sum()

deal_id                    0
deal_date                  0
deal_year                  0
firm_name                  0
firm_id                    0
round_total_raw            0
round_total                0
num_investors              0
avg_invest_per_investor    0
investor_name              0
deal_type                  0
country                    0
city                       0
year_founded               0
codex_category             0
market                     7
problem_1                  7
problem_2                  7
problem_3                  7
analytics                  7
ai                         7
point_integrated           7
firm_age                   0
dtype: int64

There are 8 records that didn't get any firm matches when doing the merge.  Let's check them out.  **Note:** I fixed 1 of the missing values above in (Fix Typo)

In [15]:
df[df.market.isnull()]

,deal_id,deal_date,deal_year,firm_name,firm_id,round_total_raw,round_total,num_investors,avg_invest_per_investor,investor_name,deal_type,country,city,year_founded,codex_category,market,problem_1,problem_2,problem_3,analytics,ai,point_integrated,firm_age
19,DEAL0014,2006-10-25,2006,IntellinX,FIRM0061,above,350000.0,3,1.166667e+05,Ascend Technology Ventures,Series A,Israel,Tel Aviv,2005,Legal Practice Management,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
20,DEAL0014,2006-10-25,2006,IntellinX,FIRM0061,4000000,4000000.0,3,1.333333e+06,Elran Technologies,Series A,Israel,Tel Aviv,2005,Legal Practice Management,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
21,DEAL0014,2006-10-25,2006,IntellinX,FIRM0061,above,4000000.0,3,1.333333e+06,SPL Software,Series A,Israel,Tel Aviv,2005,Legal Practice Management,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
280,DEAL0193,2014-01-05,2014,CellBreaker,FIRM0074,71000,71000.0,1,7.100000e+04,NC Idea,Grant,USA,North Carolina,2012,E-Discovery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
431,DEAL0285,2015-01-02,2015,CellBreaker,FIRM0074,100000,100000.0,1,1.000000e+05,500 Startups,Convertible Note,USA,North Carolina,2011,E-Discovery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
477,DEAL0317,2015-04-20,2015,CellBreaker,FIRM0074,100000,100000.0,1,1.000000e+05,undisclosed,Seed Fund,USA,North Carolina,2014,E-Discovery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
502,DEAL0328,2015-07-08,2015,CellBreaker,FIRM0074,125000,125000.0,1,1.250000e+05,undisclosed,Convertible Note,USA,North Carolina,2008,E-Discovery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7


**Notes:** ``IntellinX`` and ``CellBreaker`` are missing because we deleted those records from our firm data.  We did not have a problem categorization for those firms and upon further inspection found that they were not really related to legal tech.  We can drop all rows associated with those 2 firms.<br>
<br>
There is 1 record with firm_name ``Lawyaw`` that is also missing.  Upon further inspection, Lawyaw exists in the firms dataframe, but the ``firm_id`` is incorrect.  Nothing exists for ``FIRM0657``.  Lawyaw's firm_id is actually ``FIRM0843``.  I will fix this above the merge.  Drop other missing columns

In [16]:
df.shape

(1133, 23)

## Drop the remaining 7 rows with missing data

In [17]:
df.dropna(inplace=True)
print(df.shape)

(1126, 23)


## Export the cleansed file

In [18]:
! ls ../../data/interim

datasheet_1.csv      firms.csv            investors.csv
deals.csv            funding_investor.csv


In [19]:
file_path = '../../data/interim/investors.csv'
df.to_csv(file_path)

In [20]:
df.head(5)

,deal_id,deal_date,deal_year,firm_name,firm_id,round_total_raw,round_total,num_investors,avg_invest_per_investor,investor_name,deal_type,country,city,year_founded,codex_category,market,problem_1,problem_2,problem_3,analytics,ai,point_integrated,firm_age
0,DEAL0001,2000-03-22,2000,LexisNexis,FIRM0002,30000000,30000000.0,5,6000000.0,Actua Corporation,Venture Capital,USA,New York,1973,Legal Research,Corporate,Legal Research,Other Practice Tech,NONE,1.0,0.0,Integrated Solution,27
1,DEAL0001,2000-03-22,2000,LexisNexis,FIRM0002,above,30000000.0,5,6000000.0,Commerce One,Venture Capital,USA,New York,1973,Legal Research,Corporate,Legal Research,Other Practice Tech,NONE,1.0,0.0,Integrated Solution,27
2,DEAL0001,2000-03-22,2000,LexisNexis,FIRM0002,above,30000000.0,5,6000000.0,Kaplan,Venture Capital,USA,New York,1973,Legal Research,Corporate,Legal Research,Other Practice Tech,NONE,1.0,0.0,Integrated Solution,27
3,DEAL0001,2000-03-22,2000,LexisNexis,FIRM0002,above,30000000.0,5,6000000.0,Schoolhouse Partners,Venture Capital,USA,New York,1973,Legal Research,Corporate,Legal Research,Other Practice Tech,NONE,1.0,0.0,Integrated Solution,27
4,DEAL0001,2000-03-22,2000,LexisNexis,FIRM0002,above,30000000.0,5,6000000.0,WR Hambrecht,Venture Capital,USA,New York,1973,Legal Research,Corporate,Legal Research,Other Practice Tech,NONE,1.0,0.0,Integrated Solution,27
